#Imports

In [1]:
import librosa
import audioread
import numpy as np
#import scipy
import collections
import matplotlib.pyplot as plt
import os.path
#import IPython.display
%matplotlib inline

C:\Users\Arnulf\Anaconda3\lib\site-packages\librosa\core\audio.py:37: UserWarning: Could not import scikits.samplerate. Falling back to scipy.signal
  warnings.warn('Could not import scikits.samplerate. '


#Definitions

In [8]:
filename = "C:\\Users\\Arnulf\\Music\\Enjoy of SHINGO NAKAMURA.m4a"
sr = 16000
framesize = 2**12
n_blocks = 64
feature_dim = 84

kernel_size = 40

feature_buffer = collections.deque() #maxlen=horizon * n_blocks

#Functions

In [3]:
def get_audiofile_duration(file):
    with audioread.audio_open(file) as f:
        return f.duration

In [5]:
def load(path, sr=None, mono=True, offset=0.0, duration=None,
         dtype=np.float32):
    
    y = []
    with audioread.audio_open(os.path.realpath(path)) as input_file:
        sr_native = input_file.samplerate
        n_channels = input_file.channels

        s_start = int(np.round(sr_native * offset)) * n_channels

        if duration is None:
            s_end = np.inf
        else:
            s_end = s_start + (int(np.round(sr_native * duration))
                               * n_channels)

        n = 0

        for frame in input_file:
            frame = librosa.util.buf_to_float(frame, dtype=dtype)
            n_prev = n
            n = n + len(frame)

            if n < s_start:
                # offset is after the current frame
                # keep reading
                continue

            if s_end < n_prev:
                # we're off the end.  stop reading
                break

            if s_end < n:
                # the end is in this frame.  crop.
                frame = frame[:s_end - n_prev]

            if n_prev <= s_start <= n:
                # beginning is in this frame
                frame = frame[(s_start - n_prev):]

            # tack on the current frame
            y.append(frame)

    if y:
        y = np.concatenate(y)

        if n_channels > 1:
            y = y.reshape((-1, 2)).T
            if mono:
                y = librosa.core.to_mono(y)

        if sr is not None:
            y = librosa.core.resample(y, sr_native, sr)

        else:
            sr = sr_native

    # Final cleanup for dtype and contiguity
    y = np.ascontiguousarray(y, dtype=dtype)

    return (y, sr)

In [9]:
def create_kernel(size):
    kernel = np.zeros([2*kernel_size+1, 2*kernel_size+1], dtype='int')
    kernel[:kernel_size, :kernel_size] = 1
    kernel[kernel_size+1:, kernel_size+1:] = 1
    kernel[kernel_size+1:, :kernel_size] = -1
    kernel[:kernel_size, kernel_size+1:] = -1
    return kernel

In [10]:
def correlate2d_diagonal(matrix, kernel):
    assert np.shape(matrix)[0] == np.shape(matrix)[1]
    assert np.shape(kernel)[0] == np.shape(kernel)[1]
    length = np.shape(matrix)[0]
    L = np.shape(kernel)[0]//2
    
    result = np.zeros(length)
    for i in range(length):
        for k in range(-L, L):
            if i+k < 0 or i+k >= length:
                break
            for l in range(-L, L):
                if i+l < 0 or i+l >= length:
                    break
                result[i] = result[i] + matrix[i+l, i+k] * kernel[L+l, L+k]
    return result

# Main Program

In [7]:
blocksize = n_blocks * 1/sr * framesize #seconds

horizon = 32 #* blocksize 

%time duration = get_audiofile_duration(filename)
segments = np.arange(0,600,blocksize) # replace fixed value with duration!

kernel = create_kernel(kernel_size)

# iterate through the whole file
for i, block in enumerate(segments):   
    y = load(filename, sr=sr, mono=True, offset=block, duration=blocksize)[0]
    # constant Q transform
    CQ = librosa.core.cqt(y, sr=sr, hop_length=framesize, fmin=None, n_bins=feature_dim, bins_per_octave=12, tuning=None, resolution=2, aggregate=None, norm=1, sparsity=0.01)
#    print("CQ shape:", np.shape(CQ))
    feature_buffer.extend(CQ.T)

    novelty = correlate2d_diagonal(R_CQ, kernel)

#     print("feature buffer:", len(feature_buffer))        
#     print("feature buffer pop shape:", np.shape(feature_buffer.popleft()))
#    feature_buffer.pop
    # recurrence matrix
#    R_CQ = recurrence_matrix(CQ, k=None, width=1, metric='sqeuclidean', sym=True, axis=-1)
#    print(np.shape(R_CQ))

Wall time: 366 ms
CQ shape: (84, 65)
CQ shape: (84, 65)
CQ shape: (84, 65)
CQ shape: (84, 65)


In [ ]:
plt.figure(figsize=(12,4))
plt.plot(novelty)
plt.ylabel('Novelty')
plt.show()